In [1]:
# from sklearnex import patch_sklearn
# patch_sklearn()

In [1]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.pipeline import Pipeline

from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

In [2]:
def load_images_from_folder(folder):
    images=[]
    for filename in os.listdir(folder):
        full_path = folder + filename
        images.append(full_path)
    return images

path = '../data/three_class/test/testX/'
images_folder = sorted(load_images_from_folder(path))
print(np.size(images_folder), images_folder[:10])

2121 ['../data/three_class/test/testX/xxx00001.jpg', '../data/three_class/test/testX/xxx00002.jpg', '../data/three_class/test/testX/xxx00003.jpg', '../data/three_class/test/testX/xxx00004.jpg', '../data/three_class/test/testX/xxx00005.jpg', '../data/three_class/test/testX/xxx00006.jpg', '../data/three_class/test/testX/xxx00007.jpg', '../data/three_class/test/testX/xxx00008.jpg', '../data/three_class/test/testX/xxx00009.jpg', '../data/three_class/test/testX/xxx00010.jpg']


In [3]:
preproc = SkinLesionPreprocessing()
cfe = FeaturesExtraction(levels=['global'])
print(len(cfe.features_names))

143


In [4]:
features = []
file_names = []
for idx, img_path in tqdm(enumerate(images_folder), total=len(images_folder)):
    img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    file_names.append(img_path.split('/')[-1])
    height, width, ch = img.shape
    img_resized = cv2.resize(img,(int(width/2),int(height/2)), interpolation=cv2.INTER_AREA)
    img_preproc = preproc.preprocess(img_resized)
    features.append(cfe.extract_features(img_preproc))


df = pd.DataFrame(features, columns=cfe.features_names)
df['file_name'] = file_names
df.to_feather('../data/three_class/test_all_global_feat.f') 

100%|██████████| 2121/2121 [25:29<00:00,  1.39it/s]


In [3]:
preproc = SkinLesionPreprocessing()
cfe = FeaturesExtraction(levels=['local'], lbp_params={}, glcm_params={}, gabor_params={})
print(len(cfe.features_names))

60


In [4]:
features = []
file_names = []
for idx, img_path in tqdm(enumerate(images_folder), total= len(images_folder)):
    img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    file_names.append(img_path.split('/')[-1])
    height, width, ch = img.shape
    img_resized = cv2.resize(img,(int(width/2),int(height/2)), interpolation=cv2.INTER_AREA)
    img_preproc = preproc.preprocess(img_resized)
    mask = preproc.get_seg_mask(img_preproc)
    features.append(cfe.extract_features(img_preproc, mask))


df = pd.DataFrame(features, columns=cfe.features_names)
df['file_name'] = file_names
df.to_feather('../data/three_class/test_color_local_feat.f') 

100%|██████████| 2121/2121 [12:57<00:00,  2.73it/s]


In [22]:
# test_df_1 = pd.read_feather("../data/binary/test_all_global_feat.f")
# test_df_2 = pd.read_feather("../data/binary/test_color_local_feat.f")

# test_df = pd.concat([test_df_2.iloc[:,:60], test_df_1], axis=1)
# test_shape = test_df.shape
# file_names_df = test_df_1.iloc[:,-1]

# X_test = test_df.iloc[:, :(test_shape[1]-1)].to_numpy(dtype=np.float32)

# # here goes your classifier
# y_pred = classifier.predict(X_test)

# test_results = pd.DataFrame({'file_name': list(file_names_df), 'pred': list(y_pred)})
# test_results.to_excel('binary_test_pred.xlsx')
